<a href="https://colab.research.google.com/github/Spencer166/Capstone/blob/main/Step_11_Containerized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, ConfusionMatrixDisplay
import joblib

In [3]:
# Path to the dataset
file_path = "/content/drive/MyDrive/train.csv"

# Load into DataFrame
df_full = pd.read_csv(file_path)

# Sample 100,000 rows
df = df_full.sample(n=100000, random_state=1)

# Drop id column
df = df.drop('id', axis=1)

In [4]:
# Define Target
target = "loan_paid_back"

# Split into features and Target
X = df.drop(columns=[target])
y = df[target]

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)

# Separate numeric and categorical columns
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns


# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(drop="first",handle_unknown='ignore'), cat_cols)])

In [5]:
# Gradient Boosting model with fixed hyperparameters
gb_model = GradientBoostingClassifier(
    learning_rate=0.1,
    max_depth=4,
    n_estimators=200,
    random_state=42)


# Create pipeline with Gradient Boosting Classifier
model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", gb_model)])

# Fit to the training data
model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount',
       'interest_rate'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  Index(['gender', 'marital_status', 'education_level', 'employment_status',
       'loan_purpose', 'grade_subgrade'],
      dtype='object'))])),
                ('model',
                 GradientBoostingClassifier(max_depth=4, n_estimators=200,
                                            random_state=42))])

In [6]:
# Define predictions and probabilities
y_prob = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

print(y_prob[0:10])
print(y_pred[0:10])

[0.60389049 0.96610809 0.96032962 0.95561408 0.96207991 0.41510379
 0.91519273 0.95394607 0.9342481  0.97416745]
[1. 1. 1. 1. 1. 0. 1. 1. 1. 1.]


In [7]:
joblib.dump(model, "gradient_boosting_pipeline.joblib")

['gradient_boosting_pipeline.joblib']

In [7]:
!pip install flask pyngrok joblib

In [9]:
from flask import Flask, request, render_template_string
import traceback

app = Flask(__name__)

# Simple HTML template (embedded for Colab)
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Credit Risk Prediction</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f6f8;
        }
        .container {
            width: 480px;
            margin: 40px auto;
            padding: 20px;
            background: white;
            border-radius: 8px;
            box-shadow: 0px 0px 12px rgba(0,0,0,0.1);
        }
        h2 {
            text-align: center;
        }
        label {
            margin-top: 10px;
            display: block;
            font-weight: bold;
        }
        input, select, button {
            width: 100%;
            padding: 8px;
            margin-top: 5px;
        }
        button {
            margin-top: 15px;
            background-color: #007BFF;
            color: white;
            border: none;
            cursor: pointer;
        }
        .result {
            margin-top: 20px;
            padding: 10px;
            background-color: #eef;
            border-radius: 5px;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Credit Risk Prediction</h2>

        <form method="POST" action="/predict">

            <label>Annual Income</label>
            <input type="number" name="annual_income" required>

            <label>Debt-to-Income Ratio</label>
            <input type="number" step="0.01" name="debt_to_income_ratio" required>

            <label>Credit Score</label>
            <input type="number" name="credit_score" required>

            <label>Loan Amount</label>
            <input type="number" name="loan_amount" required>

            <label>Interest Rate</label>
            <input type="number" step="0.01" name="interest_rate" required>

            <label>Gender</label>
            <select name="gender" required>
                <option value="">Select</option>
                <option value="Male">Male</option>
                <option value="Female">Female</option>
                <option value="Other">Other</option>
            </select>

            <label>Marital Status</label>
            <select name="marital_status" required>
                <option value="">Select</option>
                <option value="Single">Single</option>
                <option value="Married">Married</option>
                <option value="Divorced">Divorced</option>
            </select>

            <label>Education Level</label>
            <select name="education_level" required>
                <option value="">Select</option>
                <option value="High School">High School</option>
                <option value="Bachelor">Bachelor</option>
                <option value="Master">Master</option>
                <option value="PhD">PhD</option>
            </select>

            <label>Employment Status</label>
            <select name="employment_status" required>
                <option value="">Select</option>
                <option value="Employed">Employed</option>
                <option value="Self-Employed">Self-Employed</option>
                <option value="Unemployed">Unemployed</option>
            </select>

            <label>Loan Purpose</label>
            <select name="loan_purpose" required>
                <option value="">Select</option>
                <option value="Debt Consolidation">Debt Consolidation</option>
                <option value="Home Improvement">Home Improvement</option>
                <option value="Education">Education</option>
                <option value="Medical">Medical</option>
                <option value="Other">Other</option>
            </select>

            <label>Grade / Subgrade</label>
            <input type="text" name="grade_subgrade" placeholder="e.g., B3" required>

            <button type="submit">Predict Credit Risk</button>
        </form>

        {% if prediction is not none %}
        <div class="result">
            <p>Prediction: {{ prediction }}</p>
            <p>Payback Probability: {{ probability }}</p>
        </div>
        {% endif %}

    </div>
</body>
</html>
"""

@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_TEMPLATE, prediction=None)

@app.route("/predict", methods=["POST"])
def predict():
    try:
        # -----------------------------
        # 1. Read form inputs
        # -----------------------------
        input_data = {
            "annual_income": float(request.form["annual_income"]),
            "debt_to_income_ratio": float(request.form["debt_to_income_ratio"]),
            "credit_score": float(request.form["credit_score"]),
            "loan_amount": float(request.form["loan_amount"]),
            "interest_rate": float(request.form["interest_rate"]),
            "gender": request.form["gender"],
            "marital_status": request.form["marital_status"],
            "education_level": request.form["education_level"],
            "employment_status": request.form["employment_status"],
            "loan_purpose": request.form["loan_purpose"],
            "grade_subgrade": request.form["grade_subgrade"]
        }

        # -----------------------------
        # 2. Convert to DataFrame
        # -----------------------------
        input_df = pd.DataFrame([input_data])

        # -----------------------------
        # 3. Model inference
        # -----------------------------
        prediction = model.predict(input_df)[0]
        probability = model.predict_proba(input_df)[0][1]

        result_text = (
            "Repay"
            if prediction == 1
            else "Default"
        )

        # -----------------------------
        # 4. Render result
        # -----------------------------
        return render_template_string(
            HTML_TEMPLATE,
            prediction=result_text,
            probability=round(float(probability), 3)
        )

    except Exception:
        # -----------------------------
        # 5. Detailed error output
        # -----------------------------
        return (
            f"<h3>Prediction Error</h3>"
            f"<pre>{traceback.format_exc()}</pre>",
            500
        )
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)

In [11]:
import getpass
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


In [12]:
from pyngrok import ngrok

ngrok.kill()

public_url = ngrok.connect(5000)
print("Public URL:", public_url)

app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

Public URL: NgrokTunnel: "https://unlignified-nonfamily-natalee.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Jan/2026 06:30:52] "GET / HTTP/1.1" 200 -


In [13]:
# Base image
FROM python:3.12-slim

# Set working directory
WORKDIR /app

# Copy dependency file
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy app code and model
COPY app.py .
COPY gradient_boosting_pipeline.joblib .

# Expose Flask port
EXPOSE 5000

# Start Flask app
CMD ["python", "app.py"]

SyntaxError: invalid syntax (ipython-input-1634223042.py, line 2)

In [ ]:
#Build the Docker Image
docker build -t credit-risk-app .

In [ ]:
#Run the Docker Container
docker run -p 5000:5000 credit-risk-app

When trying to run step 4 I'm receiving an error with this explanation:The SyntaxError: invalid syntax occurs because you're trying to execute Dockerfile commands directly within a Python code cell. Colab interprets the content of code cells as Python code, and Dockerfile syntax is not valid Python. To use these commands, you would typically need to save them in a file named Dockerfile and then use shell commands like docker build in a separate cell, but direct Docker operations might be limited in the standard Colab environment.

That same cell is taking a long time to run and I'm getting a message that says: RuntimeError:

Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.